# Chapter 5: Support Vector Machines

## 5.1 Linear SVM Classification

### 5.1.1 Soft Margin Classification

## 5.2 Nonlinear SVM Classification

### 5.2.1 Polynomial Kernel

### 5.2.2 Similarity Features

### 5.2.3 Gaussian RBF Kernel

### 5.2.4 Computational Complexity

## 5.3 SVM Regression

## 5.4 Under the Hood

### 5.4.1 Decision Function and Predictions

### 5.4.2 Training Objective

### 5.4.3 Quadratic Programming

### 5.4.4 The Dual Problem

### 5.4.5 Kernelized SVMs

### 5.4.6 Online SVMs